In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
janken_array = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [30]:
janken_class = ['グー','チョキ','パー']

In [31]:
#過去何回分の手を覚えているか
n = 3

In [32]:
Jprev = np.zeros(3*n*2)

In [48]:
#過去の手をランダムに初期化
for i in range(2*n):
    j = np.random.randint(0,3)
    Jprev[3*i:3*i+3] = janken_array[j]

In [51]:
#現在の手をランダムに初期化
j = janken_array[np.random.randint(0,3)]

In [52]:
Jprev_set = np.array([Jprev])
Jnow_set = np.array([j])

In [53]:
Jnow_set

array([[0, 0, 1]])

In [11]:
Jprev_set.shape

(1, 18)

In [12]:
Jprev_set.reshape(n,3,2)

array([[[0., 0.],
        [1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 0.],
        [1., 0.]]])

In [15]:
Jprev_set.flatten().shape

(18,)

In [24]:
def create_model():
    #モデルの初期化
    model = keras.Sequential()
    
    model.add(keras.layers.Dense(units=50,
                                input_dim=Jprev_set.flatten().shape[0],
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='tanh'))
    
    model.add(keras.layers.Dense(units=50,
                                input_dim=50,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='tanh'))
    
    model.add(keras.layers.Dense(units=3,
                                input_dim=50,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='softmax'))
    
    optimizer = keras.optimizers.Adam()
    
    model.compile(optimizer=optimizer,loss='categorical_crossentropy')
    
    
    return model

In [25]:
model = create_model()

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 50)                950       
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 153       
Total params: 3,653
Trainable params: 3,653
Non-trainable params: 0
_________________________________________________________________


In [48]:
jnow_set.shape

(1,)

In [49]:
history = model.fit(Jprev_set.reshape(3,n,2),jnow_set)

ValueError: Error when checking input: expected flatten_input to have 3 dimensions, but got array with shape (1, 18)